# 匯入 TED 資料
資料來源：https://wit3.fbk.eu/mono.php?release=XML_releases&tinfo=cleanedhtml_ted <br>
收錄各國語言的 TED 演講

In [1]:
import xml.etree.ElementTree as et
tree = et.ElementTree(file='ted_zh-tw-20160408.xml')

In [2]:
doc_list = []
for elem in tree.iter(tag='content'):
    doc_list.append(elem.text)

# 中文預處理

In [3]:
punctuation = set()
with open ('punctuation_zh_tw.txt', 'r') as f:
    for line in f.readlines():
        punctuation.add(line.strip())
with open ('punctuation_en.txt', 'r') as f:
    for line in f.readlines():
        punctuation.add(line.strip())
punctuation.add('\n')
punctuation.add(' ')

stopwords = set()
with open ('stopwords_zh_tw.txt', 'r') as f:
    for line in f.readlines():
        stopwords.add(line.strip())

In [4]:
import jieba
import jieba.posseg as pseg
jieba.load_userdict("dict.txt.big.tw")
doc_wordlist = dict()
for i in range(len(doc_list)):
    words = [word for word in jieba.cut(doc_list[i]) if word not in punctuation and word not in stopwords]
    doc_wordlist.update({i:" ".join(words)})

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/fm/3k9k8cq55zn_bw88_6fybf300000gn/T/jieba.cache
Loading model cost 0.550 seconds.
Prefix dict has been built successfully.


# 文本向量化（使用基於詞頻的詞袋模型）
<b>CountVectorizer(max_df, min_df, max_features, stop_words)</b>
- max_df: 丟棄出現在高於多少比例(float)或數量(int)的文件中之單詞（corpus-specific stop words）
- min_df: 丟棄出現在低於多少比例(float)或數量(int)的文件中之單詞
- max_features: 丟棄單詞後，最多使用多少單詞來建模（頻率由高至低排序）
- stop_words: 'english' 或輸入一個 stop_words list<br>

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = list(doc_wordlist.values())
cv = CountVectorizer(min_df=0.01, max_df=0.5, max_features=10000)
cv_features = cv.fit_transform(corpus)

In [6]:
print(cv_features)

  (0, 1787)	11
  (0, 6449)	3
  (0, 5029)	1
  (0, 6400)	1
  (0, 6419)	3
  (0, 2031)	34
  (0, 1082)	2
  (0, 5178)	1
  (0, 2384)	1
  (0, 5185)	1
  (0, 2395)	2
  (0, 184)	3
  (0, 5044)	3
  (0, 1903)	2
  (0, 5013)	1
  (0, 5866)	7
  (0, 2905)	13
  (0, 3533)	11
  (0, 6528)	1
  (0, 622)	2
  (0, 790)	1
  (0, 3401)	2
  (0, 3859)	5
  (0, 5592)	2
  (0, 4763)	1
  :	:
  (2012, 1651)	1
  (2012, 4044)	1
  (2012, 807)	1
  (2012, 4246)	1
  (2012, 2835)	1
  (2012, 5773)	1
  (2012, 3571)	1
  (2012, 7115)	1
  (2012, 445)	1
  (2012, 7680)	1
  (2012, 4221)	1
  (2012, 5768)	1
  (2012, 2988)	1
  (2012, 4219)	1
  (2012, 689)	1
  (2012, 4113)	1
  (2012, 5202)	1
  (2012, 7250)	1
  (2012, 6070)	2
  (2012, 656)	1
  (2012, 6227)	1
  (2012, 4865)	1
  (2012, 767)	1
  (2012, 2875)	1
  (2012, 1800)	1


# 文本相似度計算

## 歐幾里德距離

In [7]:
from sklearn.metrics.pairwise import euclidean_distances
doc_eu_dis = euclidean_distances(cv_features)
print(doc_eu_dis)

[[ 0.         63.61603571 82.12186067 ... 65.82552696 87.9090439
  81.70679286]
 [63.61603571  0.         62.21736092 ... 40.49691346 72.00694411
  70.04998216]
 [82.12186067 62.21736092  0.         ... 65.7951366  88.09086218
  84.27336471]
 ...
 [65.82552696 40.49691346 65.7951366  ...  0.         73.13685801
  71.39327699]
 [87.9090439  72.00694411 88.09086218 ... 73.13685801  0.
  91.90212185]
 [81.70679286 70.04998216 84.27336471 ... 71.39327699 91.90212185
   0.        ]]


In [8]:
doc_eu_dis[0]

array([ 0.        , 63.61603571, 82.12186067, ..., 65.82552696,
       87.9090439 , 81.70679286])

argsort() 預設由小排到大 <br>
argsort()[::-1] 由大排到小

In [9]:
doc_eu_dis[1].argsort()

array([   1, 1747,  226, ...,  330, 1584, 1356])

In [10]:
doc_list[1]

'我是紡織藝術家， 開創了針織塗鴉而廣為人知。 針織塗鴉是把針織或鉤針的材料 以塗鴉的方式帶入城市環境裡。 或是，更準確地說 沒有得到容許和批准。\n但是在10年前， 我開始這個活動的時候， 我沒有給它命名， 我沒有什麼野心的主張， 我沒有什麼宏偉的遠見。 我只是想要看到一些 溫暖、毛茸茸、人性化的東西 纏綣每天看起來冷冰、鋼筋、 灰濛濛的那些表面。\n所以我裹起了門的把手。 將它命名為始初的作品。 但從沒有想過，這樣的 一件小物改寫了我的人生。\n這回事引起的回應非常有趣。 我很好奇，心想： 「我還能做些什麼呢？」 要是我在公共場所這樣做， 會引起同樣的回應嗎？ 所以我裹起了住家附近的 一支停車標示杆。 這引發了狂熱的回應。 大家會把車子停好， 走下車來，盯著這杆， 搔著自己的頭，盯著這杆， 拍下這杆並且人杆合照， 這些都讓我感到很興奮， 所以我意圖把附近每一支 停車標示杆都裹起來。 我裹得越多，人們的反應就越強烈。\n因此在這刻，我傾倒了。 好過癮喲。 這一切都無比吸引我。 我有了新的愛好， 整個城市環境就是我的遊樂場。\n這些是我早期的作品。 我對這個主意很好奇： 就是去美化日常中平凡無奇， 甚至是醜陋的事物， 在保留其自身特徵或是功能的情況下， 給它量身編織一件優質的外衣。 這對我來說很有趣。 能夠把那些老氣橫秋的物品 轉化成為生氣盎然，多有趣。\n所以呢... 我想大家都看出了其中的幽默， 但是...\n（笑聲）\n我當時希望更認真推展針織塗鴉， 我希望去分析針織塗鴉這行為。 我想了解自己為什麼容許 針織塗鴉主宰我的生活、 為什麼我會這樣沉醉、 為什麼人們會有這樣熱切的回應。 我從中頓悟了一些事。 我們都生活在這個 瞬息萬變的數位世界， 但是我們仍然熱中 並渴望一些聯繫和聯結。\n我想我們都住在過度發展的都市裡， 環境讓我們都變得麻木， 過於習慣那些大型告示板、廣告牌、 和大型停車場， 麻木得都已經不再抱怨了。 所以當你無意間遇上 一支裹上了針織物的停車標示杆時， 看起來好不搭嘎， 但是慢慢地、奇怪地， 你會找到與它的關連， 就是那瞬間。 那是我愛的瞬間 那是我愛和其他人一起分享的瞬間。\n那個時候，我的好奇愈來愈強。 從消防栓和停車標示杆 到我如何能更善用針織物料， 然後再進一步做大規模、 難以駕馭的物品嗎？\n所以，這輛公車就誕生了。 這是我的轉捩點。

In [11]:
doc_list[1747]

'大家好 我們要展示何謂創作 我會重現蘭恩的一件摺紙作品 就用這張紙 所有的褶痕都在上面了 在我摺的同時 魯法斯會即興創作 用他的招牌 五弦電子大提琴表演 絕對讓各位耳目一新 準備好了嗎？要開始了！ 這只是要讓過程更有趣\n好了 你把它拿走吧\n(音樂)\n好了，開始囉 (笑聲) (鼓掌)'

## Cosine Similarity

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
doc_cosine_sim = cosine_similarity(cv_features)
print(doc_cosine_sim)

[[1.         0.07046664 0.04099549 ... 0.12180331 0.09167829 0.19597802]
 [0.07046664 1.         0.02573989 ... 0.06555155 0.06740442 0.08965652]
 [0.04099549 0.02573989 1.         ... 0.0500094  0.05336735 0.11141824]
 ...
 [0.12180331 0.06555155 0.0500094  ... 1.         0.13612085 0.14822969]
 [0.09167829 0.06740442 0.05336735 ... 0.13612085 1.         0.1060804 ]
 [0.19597802 0.08965652 0.11141824 ... 0.14822969 0.1060804  1.        ]]


In [13]:
doc_cosine_sim[1].argsort()[::-1]

array([   1, 1034,   83, ..., 1867,  226, 1709])

In [14]:
doc_list[1]

'我是紡織藝術家， 開創了針織塗鴉而廣為人知。 針織塗鴉是把針織或鉤針的材料 以塗鴉的方式帶入城市環境裡。 或是，更準確地說 沒有得到容許和批准。\n但是在10年前， 我開始這個活動的時候， 我沒有給它命名， 我沒有什麼野心的主張， 我沒有什麼宏偉的遠見。 我只是想要看到一些 溫暖、毛茸茸、人性化的東西 纏綣每天看起來冷冰、鋼筋、 灰濛濛的那些表面。\n所以我裹起了門的把手。 將它命名為始初的作品。 但從沒有想過，這樣的 一件小物改寫了我的人生。\n這回事引起的回應非常有趣。 我很好奇，心想： 「我還能做些什麼呢？」 要是我在公共場所這樣做， 會引起同樣的回應嗎？ 所以我裹起了住家附近的 一支停車標示杆。 這引發了狂熱的回應。 大家會把車子停好， 走下車來，盯著這杆， 搔著自己的頭，盯著這杆， 拍下這杆並且人杆合照， 這些都讓我感到很興奮， 所以我意圖把附近每一支 停車標示杆都裹起來。 我裹得越多，人們的反應就越強烈。\n因此在這刻，我傾倒了。 好過癮喲。 這一切都無比吸引我。 我有了新的愛好， 整個城市環境就是我的遊樂場。\n這些是我早期的作品。 我對這個主意很好奇： 就是去美化日常中平凡無奇， 甚至是醜陋的事物， 在保留其自身特徵或是功能的情況下， 給它量身編織一件優質的外衣。 這對我來說很有趣。 能夠把那些老氣橫秋的物品 轉化成為生氣盎然，多有趣。\n所以呢... 我想大家都看出了其中的幽默， 但是...\n（笑聲）\n我當時希望更認真推展針織塗鴉， 我希望去分析針織塗鴉這行為。 我想了解自己為什麼容許 針織塗鴉主宰我的生活、 為什麼我會這樣沉醉、 為什麼人們會有這樣熱切的回應。 我從中頓悟了一些事。 我們都生活在這個 瞬息萬變的數位世界， 但是我們仍然熱中 並渴望一些聯繫和聯結。\n我想我們都住在過度發展的都市裡， 環境讓我們都變得麻木， 過於習慣那些大型告示板、廣告牌、 和大型停車場， 麻木得都已經不再抱怨了。 所以當你無意間遇上 一支裹上了針織物的停車標示杆時， 看起來好不搭嘎， 但是慢慢地、奇怪地， 你會找到與它的關連， 就是那瞬間。 那是我愛的瞬間 那是我愛和其他人一起分享的瞬間。\n那個時候，我的好奇愈來愈強。 從消防栓和停車標示杆 到我如何能更善用針織物料， 然後再進一步做大規模、 難以駕馭的物品嗎？\n所以，這輛公車就誕生了。 這是我的轉捩點。

In [15]:
doc_list[1034]

'我想要跟你們講一個故事 我大部分的時間 在教導成人如何使用視覺語言 還有在工作的地方塗鴉。 所以，很自然的我遇到很多反對的聲音， 因為塗鴉被認為是一種不智慧的表現 且對於學習有反效果。 但是，我認為那種講法是錯誤的。 因為，我知道塗鴉可以對我們的資訊處理方式 還有解決問題的方式， 有著重大的影響。\n所以我很好奇為什麼， 社會對塗鴉的認知， 與事實有著差異。 我找到了些有趣的事情。 舉例來說： 塗鴉這個字沒有個正向的意義。 在十七世紀，塗鴉指的是一個簡單的人或傻瓜－ 就像Yankee Doodle一樣。 在十八世紀，doodle成了一個動詞， 這動詞的意思指，去嘲笑別人或欺騙別人。 十九世紀，被解釋為貪污的政客。 而今天的解釋，可能是最冒犯人的解釋， 至少對我而言是如此。 塗鴉正式解釋為： 混日子， 拖拖拉拉，耍猴戲， 畫沒有意意的圖案， 做沒有價值的事情， 還有我最喜歡的解釋－ 什麼也不作。 這樣就清楚地瞭解為什麼人們對在工作時塗鴉這樣的不喜。 工作時不做事就好比似一隻米蟲一樣； 是不對的事情。\n（笑聲）\n我還聽過糟糕的故事呢。 有些人因為在課堂上塗鴉被老師責罵。 也有老闆責罵員工在開會時塗鴉。 文化上認為塗鴉是種不好的行為。 因為在課堂上，或在會議裡，我們應該學習。 很可惜的， 媒體一直強調塗鴉是不好的行為。 他們一直反面地報導塗鴉－ 像是一個重要的人在開會中塗鴉，而且 他們通常用「發現」 或是「逮到」 好比似塗鴉是個犯罪行為一般。\n再者呢，塗鴉可以從心裡學來解釋－ 這要謝謝佛洛伊德。 在三十年代，佛洛伊德提出 一個人的心態可以藉由塗鴉， 而分析出來。 這不太正確，但提卻發生在布萊爾（英國首相）身上， 在2005達沃斯論談上 布萊爾的塗鴉被「發現」了， 人們也給他貼上了這些標簽。 不過，現在我們瞭解這其實上是蓋茲的塗鴉。 （笑聲） 比爾，如果你在這的話，沒人會認為你是自大狂的。 這種的負面影響， 造成了人們不喜歡分享塗鴉。\n我認為我找到了問題所在。 我們的文化， 太注重於語言溝通， 以至於我們忽略圖像溝通的價值。 這讓我覺得不太愉快。 所以我認為那種想法應該被戳破。 我今天要跟大家講塗鴉的事實。 事實上呢： 塗鴉是種神妙的工具 是種我們需要記住和重新學習的工具。\n這是塗鴉的新解釋。 我希望在場有牛津英文辭典的代表， 因為我們等等需要好好的談談。 塗鴉

# 文本向量化（使用基於 TF-IDF 的詞袋模型）

In [16]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer()
tfidf = transformer.fit_transform(cv.fit_transform(corpus)) 
print(tfidf)

  (0, 7844)	0.02030338216506918
  (0, 7816)	0.029517390768040132
  (0, 7786)	0.012750710304175865
  (0, 7782)	0.013766396468700497
  (0, 7763)	0.020382650785291622
  (0, 7756)	0.026188077833131776
  (0, 7753)	0.021531711381848347
  (0, 7752)	0.027023369714432262
  (0, 7751)	0.025838309171928763
  (0, 7733)	0.04316206712959936
  (0, 7711)	0.031170266128290616
  (0, 7707)	0.012883436443973079
  (0, 7645)	0.08573745244866819
  (0, 7576)	0.02968438120352241
  (0, 7549)	0.017103480228176086
  (0, 7526)	0.0470356322215887
  (0, 7515)	0.01669859636750023
  (0, 7501)	0.014905495190758537
  (0, 7492)	0.01811955254431261
  (0, 7476)	0.022265627855708403
  (0, 7422)	0.026765243682817845
  (0, 7418)	0.014701045940277942
  (0, 7408)	0.028708415071253393
  (0, 7388)	0.01701575177547937
  (0, 7387)	0.015534918159923146
  :	:
  (2012, 278)	0.00763631283926993
  (2012, 272)	0.022286884025883785
  (2012, 261)	0.02420712125281728
  (2012, 247)	0.029827234301685466
  (2012, 230)	0.026754762379600085
  (20

## 歐幾里德距離

In [17]:
doc_eu_sim = euclidean_distances(tfidf)
print(doc_eu_sim)

[[0.         1.39042407 1.39851753 ... 1.34662292 1.37922475 1.3263257 ]
 [1.39042407 0.         1.40452016 ... 1.38784045 1.38575174 1.37680478]
 [1.39851753 1.40452016 0.         ... 1.39554204 1.39441826 1.35683483]
 ...
 [1.34662292 1.38784045 1.39554204 ... 0.         1.35365594 1.34700808]
 [1.37922475 1.38575174 1.39441826 ... 1.35365594 0.         1.3755845 ]
 [1.3263257  1.37680478 1.35683483 ... 1.34700808 1.3755845  0.        ]]


In [18]:
doc_eu_sim[1].argsort()

array([   1, 1034,   83, ..., 1439,  851,  659])

In [19]:
doc_list[1]

'我是紡織藝術家， 開創了針織塗鴉而廣為人知。 針織塗鴉是把針織或鉤針的材料 以塗鴉的方式帶入城市環境裡。 或是，更準確地說 沒有得到容許和批准。\n但是在10年前， 我開始這個活動的時候， 我沒有給它命名， 我沒有什麼野心的主張， 我沒有什麼宏偉的遠見。 我只是想要看到一些 溫暖、毛茸茸、人性化的東西 纏綣每天看起來冷冰、鋼筋、 灰濛濛的那些表面。\n所以我裹起了門的把手。 將它命名為始初的作品。 但從沒有想過，這樣的 一件小物改寫了我的人生。\n這回事引起的回應非常有趣。 我很好奇，心想： 「我還能做些什麼呢？」 要是我在公共場所這樣做， 會引起同樣的回應嗎？ 所以我裹起了住家附近的 一支停車標示杆。 這引發了狂熱的回應。 大家會把車子停好， 走下車來，盯著這杆， 搔著自己的頭，盯著這杆， 拍下這杆並且人杆合照， 這些都讓我感到很興奮， 所以我意圖把附近每一支 停車標示杆都裹起來。 我裹得越多，人們的反應就越強烈。\n因此在這刻，我傾倒了。 好過癮喲。 這一切都無比吸引我。 我有了新的愛好， 整個城市環境就是我的遊樂場。\n這些是我早期的作品。 我對這個主意很好奇： 就是去美化日常中平凡無奇， 甚至是醜陋的事物， 在保留其自身特徵或是功能的情況下， 給它量身編織一件優質的外衣。 這對我來說很有趣。 能夠把那些老氣橫秋的物品 轉化成為生氣盎然，多有趣。\n所以呢... 我想大家都看出了其中的幽默， 但是...\n（笑聲）\n我當時希望更認真推展針織塗鴉， 我希望去分析針織塗鴉這行為。 我想了解自己為什麼容許 針織塗鴉主宰我的生活、 為什麼我會這樣沉醉、 為什麼人們會有這樣熱切的回應。 我從中頓悟了一些事。 我們都生活在這個 瞬息萬變的數位世界， 但是我們仍然熱中 並渴望一些聯繫和聯結。\n我想我們都住在過度發展的都市裡， 環境讓我們都變得麻木， 過於習慣那些大型告示板、廣告牌、 和大型停車場， 麻木得都已經不再抱怨了。 所以當你無意間遇上 一支裹上了針織物的停車標示杆時， 看起來好不搭嘎， 但是慢慢地、奇怪地， 你會找到與它的關連， 就是那瞬間。 那是我愛的瞬間 那是我愛和其他人一起分享的瞬間。\n那個時候，我的好奇愈來愈強。 從消防栓和停車標示杆 到我如何能更善用針織物料， 然後再進一步做大規模、 難以駕馭的物品嗎？\n所以，這輛公車就誕生了。 這是我的轉捩點。

In [20]:
doc_list[1034]

'我想要跟你們講一個故事 我大部分的時間 在教導成人如何使用視覺語言 還有在工作的地方塗鴉。 所以，很自然的我遇到很多反對的聲音， 因為塗鴉被認為是一種不智慧的表現 且對於學習有反效果。 但是，我認為那種講法是錯誤的。 因為，我知道塗鴉可以對我們的資訊處理方式 還有解決問題的方式， 有著重大的影響。\n所以我很好奇為什麼， 社會對塗鴉的認知， 與事實有著差異。 我找到了些有趣的事情。 舉例來說： 塗鴉這個字沒有個正向的意義。 在十七世紀，塗鴉指的是一個簡單的人或傻瓜－ 就像Yankee Doodle一樣。 在十八世紀，doodle成了一個動詞， 這動詞的意思指，去嘲笑別人或欺騙別人。 十九世紀，被解釋為貪污的政客。 而今天的解釋，可能是最冒犯人的解釋， 至少對我而言是如此。 塗鴉正式解釋為： 混日子， 拖拖拉拉，耍猴戲， 畫沒有意意的圖案， 做沒有價值的事情， 還有我最喜歡的解釋－ 什麼也不作。 這樣就清楚地瞭解為什麼人們對在工作時塗鴉這樣的不喜。 工作時不做事就好比似一隻米蟲一樣； 是不對的事情。\n（笑聲）\n我還聽過糟糕的故事呢。 有些人因為在課堂上塗鴉被老師責罵。 也有老闆責罵員工在開會時塗鴉。 文化上認為塗鴉是種不好的行為。 因為在課堂上，或在會議裡，我們應該學習。 很可惜的， 媒體一直強調塗鴉是不好的行為。 他們一直反面地報導塗鴉－ 像是一個重要的人在開會中塗鴉，而且 他們通常用「發現」 或是「逮到」 好比似塗鴉是個犯罪行為一般。\n再者呢，塗鴉可以從心裡學來解釋－ 這要謝謝佛洛伊德。 在三十年代，佛洛伊德提出 一個人的心態可以藉由塗鴉， 而分析出來。 這不太正確，但提卻發生在布萊爾（英國首相）身上， 在2005達沃斯論談上 布萊爾的塗鴉被「發現」了， 人們也給他貼上了這些標簽。 不過，現在我們瞭解這其實上是蓋茲的塗鴉。 （笑聲） 比爾，如果你在這的話，沒人會認為你是自大狂的。 這種的負面影響， 造成了人們不喜歡分享塗鴉。\n我認為我找到了問題所在。 我們的文化， 太注重於語言溝通， 以至於我們忽略圖像溝通的價值。 這讓我覺得不太愉快。 所以我認為那種想法應該被戳破。 我今天要跟大家講塗鴉的事實。 事實上呢： 塗鴉是種神妙的工具 是種我們需要記住和重新學習的工具。\n這是塗鴉的新解釋。 我希望在場有牛津英文辭典的代表， 因為我們等等需要好好的談談。 塗鴉

## Cosine Similiarity

In [21]:
doc_cosine_sim = cosine_similarity(tfidf)
print(doc_cosine_sim)

[[1.         0.03336046 0.02207436 ... 0.09330336 0.04886955 0.12043006]
 [0.03336046 1.         0.01366157 ... 0.03694944 0.03984606 0.0522043 ]
 [0.02207436 0.01366157 1.         ... 0.0262312  0.02779886 0.07949963]
 ...
 [0.09330336 0.03694944 0.0262312  ... 1.         0.0838078  0.09278462]
 [0.04886955 0.03984606 0.02779886 ... 0.0838078  1.         0.05388364]
 [0.12043006 0.0522043  0.07949963 ... 0.09278462 0.05388364 1.        ]]


In [22]:
doc_cosine_sim[1].argsort()[::-1]

array([   1, 1034,   83, ...,  226,  659,  103])

In [23]:
doc_list[1]

'我是紡織藝術家， 開創了針織塗鴉而廣為人知。 針織塗鴉是把針織或鉤針的材料 以塗鴉的方式帶入城市環境裡。 或是，更準確地說 沒有得到容許和批准。\n但是在10年前， 我開始這個活動的時候， 我沒有給它命名， 我沒有什麼野心的主張， 我沒有什麼宏偉的遠見。 我只是想要看到一些 溫暖、毛茸茸、人性化的東西 纏綣每天看起來冷冰、鋼筋、 灰濛濛的那些表面。\n所以我裹起了門的把手。 將它命名為始初的作品。 但從沒有想過，這樣的 一件小物改寫了我的人生。\n這回事引起的回應非常有趣。 我很好奇，心想： 「我還能做些什麼呢？」 要是我在公共場所這樣做， 會引起同樣的回應嗎？ 所以我裹起了住家附近的 一支停車標示杆。 這引發了狂熱的回應。 大家會把車子停好， 走下車來，盯著這杆， 搔著自己的頭，盯著這杆， 拍下這杆並且人杆合照， 這些都讓我感到很興奮， 所以我意圖把附近每一支 停車標示杆都裹起來。 我裹得越多，人們的反應就越強烈。\n因此在這刻，我傾倒了。 好過癮喲。 這一切都無比吸引我。 我有了新的愛好， 整個城市環境就是我的遊樂場。\n這些是我早期的作品。 我對這個主意很好奇： 就是去美化日常中平凡無奇， 甚至是醜陋的事物， 在保留其自身特徵或是功能的情況下， 給它量身編織一件優質的外衣。 這對我來說很有趣。 能夠把那些老氣橫秋的物品 轉化成為生氣盎然，多有趣。\n所以呢... 我想大家都看出了其中的幽默， 但是...\n（笑聲）\n我當時希望更認真推展針織塗鴉， 我希望去分析針織塗鴉這行為。 我想了解自己為什麼容許 針織塗鴉主宰我的生活、 為什麼我會這樣沉醉、 為什麼人們會有這樣熱切的回應。 我從中頓悟了一些事。 我們都生活在這個 瞬息萬變的數位世界， 但是我們仍然熱中 並渴望一些聯繫和聯結。\n我想我們都住在過度發展的都市裡， 環境讓我們都變得麻木， 過於習慣那些大型告示板、廣告牌、 和大型停車場， 麻木得都已經不再抱怨了。 所以當你無意間遇上 一支裹上了針織物的停車標示杆時， 看起來好不搭嘎， 但是慢慢地、奇怪地， 你會找到與它的關連， 就是那瞬間。 那是我愛的瞬間 那是我愛和其他人一起分享的瞬間。\n那個時候，我的好奇愈來愈強。 從消防栓和停車標示杆 到我如何能更善用針織物料， 然後再進一步做大規模、 難以駕馭的物品嗎？\n所以，這輛公車就誕生了。 這是我的轉捩點。

In [24]:
doc_list[1034]

'我想要跟你們講一個故事 我大部分的時間 在教導成人如何使用視覺語言 還有在工作的地方塗鴉。 所以，很自然的我遇到很多反對的聲音， 因為塗鴉被認為是一種不智慧的表現 且對於學習有反效果。 但是，我認為那種講法是錯誤的。 因為，我知道塗鴉可以對我們的資訊處理方式 還有解決問題的方式， 有著重大的影響。\n所以我很好奇為什麼， 社會對塗鴉的認知， 與事實有著差異。 我找到了些有趣的事情。 舉例來說： 塗鴉這個字沒有個正向的意義。 在十七世紀，塗鴉指的是一個簡單的人或傻瓜－ 就像Yankee Doodle一樣。 在十八世紀，doodle成了一個動詞， 這動詞的意思指，去嘲笑別人或欺騙別人。 十九世紀，被解釋為貪污的政客。 而今天的解釋，可能是最冒犯人的解釋， 至少對我而言是如此。 塗鴉正式解釋為： 混日子， 拖拖拉拉，耍猴戲， 畫沒有意意的圖案， 做沒有價值的事情， 還有我最喜歡的解釋－ 什麼也不作。 這樣就清楚地瞭解為什麼人們對在工作時塗鴉這樣的不喜。 工作時不做事就好比似一隻米蟲一樣； 是不對的事情。\n（笑聲）\n我還聽過糟糕的故事呢。 有些人因為在課堂上塗鴉被老師責罵。 也有老闆責罵員工在開會時塗鴉。 文化上認為塗鴉是種不好的行為。 因為在課堂上，或在會議裡，我們應該學習。 很可惜的， 媒體一直強調塗鴉是不好的行為。 他們一直反面地報導塗鴉－ 像是一個重要的人在開會中塗鴉，而且 他們通常用「發現」 或是「逮到」 好比似塗鴉是個犯罪行為一般。\n再者呢，塗鴉可以從心裡學來解釋－ 這要謝謝佛洛伊德。 在三十年代，佛洛伊德提出 一個人的心態可以藉由塗鴉， 而分析出來。 這不太正確，但提卻發生在布萊爾（英國首相）身上， 在2005達沃斯論談上 布萊爾的塗鴉被「發現」了， 人們也給他貼上了這些標簽。 不過，現在我們瞭解這其實上是蓋茲的塗鴉。 （笑聲） 比爾，如果你在這的話，沒人會認為你是自大狂的。 這種的負面影響， 造成了人們不喜歡分享塗鴉。\n我認為我找到了問題所在。 我們的文化， 太注重於語言溝通， 以至於我們忽略圖像溝通的價值。 這讓我覺得不太愉快。 所以我認為那種想法應該被戳破。 我今天要跟大家講塗鴉的事實。 事實上呢： 塗鴉是種神妙的工具 是種我們需要記住和重新學習的工具。\n這是塗鴉的新解釋。 我希望在場有牛津英文辭典的代表， 因為我們等等需要好好的談談。 塗鴉

# 小試身手
- 比較看看使用詞頻和 TF-IDF 計算文本相似度的結果有什麼差異
- 加入 n-gram 計算相似度